In [1]:
!pip install alt-profanity-check -q
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pickle
import os
from sklearn.preprocessing import LabelEncoder

# Removing duplicate records & displaying them

In [3]:
data = pd.read_csv("/content/Data for AI Assignment.csv")

In [4]:
duplicates = data[data.duplicated(subset='Text')]
data = data.drop_duplicates(subset='Text').reset_index(drop=True)

In [5]:
#duplicate records
print(duplicates)

                                                    Text Classification
5067   i feel on the verge of tears from weariness i ...            joy
6133               i still feel a craving for sweet food           love
6563   i tend to stop breathing when i m feeling stre...          anger
7623   i was intensely conscious of how much cash i h...        sadness
7685   im still not sure why reilly feels the need to...       surprise
8246   i am not amazing or great at photography but i...           love
9596   ive also made it with both sugar measurements ...            joy
9687   i had to choose the sleek and smoother feel of...            joy
9769   i often find myself feeling assaulted by a mul...        sadness
9786        i feel im being generous with that statement            joy
10117  i feel pretty tortured because i work a job an...           fear
10581                       i feel most passionate about            joy
11273  i was so stubborn and that it took you getting...        

# Remove Records With Profanities

In [6]:
data['profanity_level']= "-"

In [7]:
from profanity_check import predict, predict_prob
for i in range(0, len(data)):
  custom_sentence = f"{data['Text'][i]}"
  p = predict_prob([custom_sentence])[0]
  data['profanity_level'][i] = round(p,3)



In [8]:
data_with_profanities =  data[data['profanity_level'] >= 0.8]

In [9]:
#data with profanities
pd.DataFrame(data_with_profanities)

,Text,Classification,profanity_level
56,i hate living under my dads roof because it gi...,sadness,0.808
80,i feel like a miserable piece of garbage,sadness,0.886
95,i feel like throwing away the shitty piece of ...,sadness,0.997
99,i say goodbye to the fam theyre all sad a cryi...,anger,0.89
189,i think im just being stupid feeling nervous,fear,0.815
...,...,...,...
17754,i feel like a moronic bastard,sadness,0.999
17855,i was stupid and said yes which made me feel i...,sadness,0.995
17866,i always feel stupid afterwards,sadness,0.998
17884,i feel so pathetic and useless being unable to...,sadness,0.808


In [10]:
#data without profanities
data = data[data['profanity_level'] < 0.8].reset_index(drop=True)
data.head(5)

In [14]:
data.to_csv("preprocessed_data.csv")

# Finetuning a BERT base model for emotion classification

In [15]:
df = pd.read_csv("preprocessed_data.csv")

In [16]:
df.columns

Index(['Unnamed: 0', 'Text', 'Classification', 'profanity_level'], dtype='object')

In [18]:
# Split the data into training, validation, and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_classes = len(df['Classification'].unique())
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Tokenize and preprocess the data
max_length = 150
train_data_tokenized = tokenizer.batch_encode_plus(
    train_data['Text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors='pt'
)

val_data_tokenized = tokenizer.batch_encode_plus(
    val_data['Text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors='pt'
)

test_data_tokenized = tokenizer.batch_encode_plus(
    test_data['Text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors='pt'
)

# Encode labels
label_encoder = LabelEncoder()
train_labels = torch.tensor(label_encoder.fit_transform(train_data['Classification'].tolist()))
val_labels = torch.tensor(label_encoder.transform(val_data['Classification'].tolist()))
test_labels = torch.tensor(label_encoder.transform(test_data['Classification'].tolist()))

# Create DataLoader
def create_data_loader(tokenized_data, labels, batch_size, shuffle=True):
    dataset = TensorDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'], labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

batch_size = 32
train_loader = create_data_loader(train_data_tokenized, train_labels, batch_size)
val_loader = create_data_loader(val_data_tokenized, val_labels, batch_size)
test_loader = create_data_loader(test_data_tokenized, test_labels, batch_size)

# Set up training parameters
epochs = 3
lr = 2e-5
optimizer = AdamW(model.parameters(), lr=lr)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct_predictions = 0
        total_samples = 0

        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            _, predictions = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(predictions == labels).item()
            total_samples += labels.size(0)

        average_val_loss = val_loss / len(val_loader)
        accuracy = correct_predictions / total_samples

        print(f'Epoch {epoch + 1}/{epochs}: Val Loss = {average_val_loss}, Accuracy = {accuracy}')

# Test the model
model.eval()
with torch.no_grad():
    test_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()
        _, predictions = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(predictions == labels).item()
        total_samples += labels.size(0)

    average_test_loss = test_loss / len(test_loader)
    accuracy = correct_predictions / total_samples

    print(f'Test Loss = {average_test_loss}, Accuracy = {accuracy}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3: Val Loss = 0.19236479998965347, Accuracy = 0.9235993208828522
Epoch 2/3: Val Loss = 0.13873679327246333, Accuracy = 0.9411431805319751
Epoch 3/3: Val Loss = 0.12923972793006605, Accuracy = 0.9371816638370118
Test Loss = 0.14240534816469466, Accuracy = 0.9320882852292021


In [19]:
#downloading the weights

model.save_pretrained('/content/weights')
label_encoder_path = '/content/weights/label_encoder.pkl'
with open(label_encoder_path, 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

In [ ]:
import shutil


folder_path = '/content/weights'
zip_file_name = '/content/weights'

shutil.make_archive(zip_file_name, 'zip', folder_path)


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
shutil.move("/content/weights.zip", "/content/drive/MyDrive/0_audios_to_train")

'/content/drive/MyDrive/0_audios_to_train/weights.zip'